In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split
from datetime import datetime

torch.manual_seed(123)
torch.set_default_dtype(torch.double)

In [3]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))
print(device)

cuda


In [4]:
def load_cifar(train_val_split=0.9, data_path='../data/', preprocessor=None):
    
    # Define preprocessor if not already given
    if preprocessor is None:
        preprocessor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4915, 0.4823, 0.4468),
                                (0.2470, 0.2435, 0.2616))
        ])
    
    # load datasets
    data_train_val = datasets.CIFAR10(
        data_path,      
        train=True,      
        download=True,
        transform=preprocessor)

    data_test = datasets.CIFAR10(
        data_path, 
        train=False,
        download=True,
        transform=preprocessor)

    # train/validation split
    n_train = int(len(data_train_val)*train_val_split)
    n_val =  len(data_train_val) - n_train

    data_train, data_val = random_split(
        data_train_val, 
        [n_train, n_val],
        generator=torch.Generator().manual_seed(123)
    )

    print("Size of the train dataset:        ", len(data_train))
    print("Size of the validation dataset:   ", len(data_val))
    print("Size of the test dataset:         ", len(data_test))
    
    return (data_train, data_val, data_test)

cifar10_train, cifar10_val, cifar10_test = load_cifar()

# Now define a lighter version of CIFAR10: cifar
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']

# For each dataset, keep only airplanes and birds
cifar2_train = [(img, label_map[label]) for img, label in cifar10_train if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]
cifar2_test = [(img, label_map[label]) for img, label in cifar10_test if label in [0, 2]]

print('Size of the training dataset: ', len(cifar2_train))
print('Size of the validation dataset: ', len(cifar2_val))
print('Size of the test dataset: ', len(cifar2_test))

Files already downloaded and verified
Files already downloaded and verified
Size of the train dataset:         45000
Size of the validation dataset:    5000
Size of the test dataset:          10000
Size of the training dataset:  9017
Size of the validation dataset:  983
Size of the test dataset:  2000


In [5]:
class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()  # to inherit the '__init__' method from the 'nn.Module' class
        # Add whatever you want here (e.g layers and activation functions)
        # The order and names don't matter here but it is easier to understand
        # if you go for Layer1, fun1, layer2, fun2, etc
        # Some conventions:
        # - conv stands for convolution
        # - pool for pooling
        # - fc for fully connected

        self.flat = nn.Flatten()
        # 32*32*3: determined by our dataset: 32x32 RGB images
        self.fc1 = nn.Linear(32*32*3, 512)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 128)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 32)
        self.act3 = nn.ReLU()
        # 2: determined by our number of classes (birds and planes)
        self.fc4 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.flat(x)
        out = self.act1(self.fc1(out))
        out = self.act2(self.fc2(out))
        out = self.act3(self.fc3(out))
        out = self.fc4(out)
        return out

In [6]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    n_batch = len(train_loader)
    losses_train = []
    model.train()
    optimizer.zero_grad(set_to_none=True)
    
    for epoch in range(1, n_epochs + 1):
        
        loss_train = 0.0
        for imgs, labels in train_loader:

            imgs = imgs.to(device=device, dtype=torch.double)
            labels = labels.to(device=device)

            outputs = model(imgs)
            
            loss = loss_fn(outputs, labels)
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()

            loss_train += loss.item()
            
        losses_train.append(loss_train / n_batch)

        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
    return losses_train

In [7]:
def train_manual_update(n_epochs, lr, model, loss_fn, train_loader, verbose = False):
    n_batch = len(train_loader)
    losses_train = []
    model.train()
    
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:

            imgs = imgs.to(device=device, dtype=torch.double)
            labels = labels.to(device=device)

            outputs = model(imgs)
            
            loss = loss_fn(outputs, labels)
            loss.backward()
            
            # optimizer step
            with torch.no_grad():
                for p in model.parameters():
                    p -= lr * p.grad
            

            with torch.no_grad():
                for p in model.parameters():
                    p.grad.zero_()

            loss_train += loss.item()
            
        losses_train.append(loss_train / n_batch)

        if epoch == 1 or epoch % 10 == 0 or verbose:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
    return losses_train

In [8]:
torch.manual_seed(123)
model = MyMLP().to(device=device) 
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
n_epochs = 10
train(n_epochs, optimizer, model, nn.CrossEntropyLoss(), train_loader)

10:48:47.420266  |  Epoch 1  |  Training loss 0.485
10:49:00.038895  |  Epoch 10  |  Training loss 0.177


[0.48534855708013924,
 0.40581430436787774,
 0.3625198246528467,
 0.3233250236449739,
 0.29195811018289874,
 0.2527437484177395,
 0.2234791375719246,
 0.2142932065058646,
 0.18786778244868008,
 0.17721166132606572]

In [11]:
torch.manual_seed(123)
model = MyMLP().to(device=device) 
train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
n_epochs = 10
train_manual_update(n_epochs, 1e-2, model, nn.CrossEntropyLoss(), train_loader, verbose=True)

10:50:20.974563  |  Epoch 1  |  Training loss 0.645
10:50:22.192655  |  Epoch 2  |  Training loss 0.533
10:50:23.384039  |  Epoch 3  |  Training loss 0.478
10:50:24.575097  |  Epoch 4  |  Training loss 0.447
10:50:25.824148  |  Epoch 5  |  Training loss 0.420
10:50:27.103720  |  Epoch 6  |  Training loss 0.397
10:50:28.311313  |  Epoch 7  |  Training loss 0.377
10:50:29.519025  |  Epoch 8  |  Training loss 0.360
10:50:30.748854  |  Epoch 9  |  Training loss 0.343
10:50:32.003120  |  Epoch 10  |  Training loss 0.326


[0.645121719744059,
 0.5329839468263902,
 0.4778253794609127,
 0.44681255832843997,
 0.4197904609343939,
 0.3968746333461311,
 0.3773816702722962,
 0.3595321877492997,
 0.34261367039377877,
 0.3259341437682193]